# Dogs vs. cats image classification: Inception V3

In [ ]:
# Handle imports up-front
import os
import sys
import glob

# Silence logging messages from TensorFlow, except errors
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# Use a specific GPU
os.environ['CUDA_VISIBLE_DEVICES']='1'

# PyPI imports
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

# Figure out if we are running on Kaggle or not, if so
# add the location of utils.py to path so we can import
path_list=os.getcwd().split(os.sep)

if path_list[1] == 'kaggle':
    sys.path.append('/kaggle/input/image-classification-functions/')

# Import custom helper functions from utils.py
from utils import prep_data
from utils import plot_single_training_run

# Silence logging messages from TensorFlow, except errors
tf.get_logger().setLevel('ERROR')

# Limit TensorFlow's CPU usage
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)

## 1. Data preparation

### 1.1. Load the data paths

In [ ]:
# Decompress and organize the images
training_data_path, testing_data_path=prep_data()

# Get lists of training dog and cat images
training_dogs=glob.glob(f'{training_data_path}/dogs/dog.*')
training_cats=glob.glob(f'{training_data_path}/cats/cat.*')

### 1.2. Create training, validation and testing datasets

In [ ]:
training_dataset, validation_dataset=tf.keras.utils.image_dataset_from_directory(
    training_data_path,
    validation_split=0.2,
    subset='both',
    seed=315,
    image_size=(256, int(256*(3/4))),
    batch_size=16
)

In [ ]:
# create the base pre-trained model
base_model=InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer, and extra dense layer
# and a binary output layer
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='relu')(x)
predictions=Dense(1, activation='sigmoid')(x)

model=Model(inputs=base_model.input, outputs=predictions)

# Train only the top layers by freezing all convolutional InceptionV3 layers
for layer in base_model.layers:
   layer.trainable = False

# Compile the model to train on binary cross-entropy loss
model.compile(optimizer='rmsprop', loss='binary_crossentropy')

# Train the model on the new data for a few epochs
training_result=model.fit(
   training_dataset,
   validation_data=validation_dataset,
   epochs=5
)

In [ ]:
# Plot the results
plot_single_training_run(training_results).show()